In [23]:
import numpy as np
import nltk.tokenize as d
import string
# Data preprocessing
#https://github.com/ryanmcdermott/trump-speeches
with open ('speeches.txt','r') as file:
    data=file.read()
# text preprocessing
table = str.maketrans({key: None for key in string.punctuation})
table = str.maketrans({key: None for key in string.digits})
table = str.maketrans({key: None for key in '\n?.!/;:[!@#$]\ufeff=–()..._…"&é%”-—‘“0123456789,'})
data = data.translate(table).lower()  
#tokenize every character
data_tk=list(data)
#get distinct character tuple
characters=set(data_tk)
char_index = [x[0] for x in characters]
#print(char_index)
char_to_index = {v:i for i,v in enumerate(characters)}
ix_to_char = { i:ch for i,ch in enumerate(characters) }
#print(char_to_index)
data_index = [char_to_index[char] for char in data]
print(char_to_index)
#print(ix_to_char)

{'w': 0, 'i': 1, 'e': 2, 'c': 3, 'f': 4, 'o': 5, ' ': 6, 's': 7, 'p': 8, 'r': 9, 'd': 10, 'x': 11, 'h': 12, "'": 13, 'z': 14, 'b': 15, 'k': 16, '’': 17, 'a': 18, 't': 19, 'm': 20, 'u': 21, 'v': 22, 'n': 23, 'g': 24, 'j': 25, 'l': 26, 'q': 27, 'y': 28}


854107

In [0]:

#RNN Architecture

unit_size=100
step_size=25
learning_rate=1e-1

#weight inilitiaze
Whx =np.random.randn(unit_size,len(characters))*0.01
Whh =np.random.randn(unit_size,unit_size)*0.01
Why =np.random.randn(len(characters),unit_size)*0.01
Bh =np.zeros(shape=(unit_size,1))
By =np.zeros(shape=(len(characters),1))

In [0]:


##forward propagation and loss


def forward_prop(inputs,targets,hprevious):

#use  h and y arrays to keep functions (tanh, softmax) outputs
    y=np.zeros(shape=(step_size,len(characters)))
    h= np.zeros(( step_size + 1,unit_size ))
    hprevious=np.squeeze(hprevious).shape
    h[-1] =np.copy(hprevious)
    total_loss=0

#one hot encoding for inputs
    x=np.zeros(shape=(len(inputs),len(characters)))

    for i in range(len(inputs)):
    #print(i)
        x[i][inputs[i]]=1

    for t in range(step_size):
    
        h[t]=np.tanh(np.dot(Whx,x[t])+np.dot(Whh,h[t-1]))#+Bh[:,0])#+Bh[:,0])
        y_linear=np.dot(Why,h[t,])+By[:,0] # linear part of softmax func
        y[t]=np.exp(y_linear - np.max(y_linear))/(np.exp(y_linear - np.max(y_linear)).sum()) # softmax func
        total_loss += -np.log(y[t][targets[t]]) # Calculate loss, loss equal to negative log sum of correct index character's probability 



##Back Propogation
#create array to keep parameters derivatives
    dLdWhx=np.zeros(Whx.shape) # loss derivatives respect to Whx, to find Whx effect to loss
    dLdWhh=np.zeros(Whh.shape)
    dLdWhy=np.zeros(Why.shape)
    dLdBy=np.zeros(By.shape)
    dLdBh=np.zeros(Bh.shape)
    dLdHp=np.zeros(h[0].shape)  # to add previous layer memory, 


    for  i in reversed (range(len(inputs))):               
        dy=np.copy(y[i]) # copy t th probability 
        dy=np.reshape(dy,(len(characters),1))
        dy[targets[i]]-=1    # derivatives respect to loss only depend on correct character prob., so change prob to equals 1
        #print(dy[targets[i]])
        dLdWhy+=np.dot(dy,np.reshape(h[i],(1,unit_size)))
        dLdBy+=dy
        dhl= np.dot(Why.T,dy)+np.expand_dims(dLdHp, axis=1) # hidden layer back propagation
        dact = (1 - np.expand_dims(h[i], axis=1) * np.expand_dims(h[i], axis=1)) * dhl # backprob tanh activation func
        dLdBh+=dact
        dLdWhx+=np.dot(dact,np.expand_dims(x[i], axis=0))
        dLdWhh += np.dot(dact,np.expand_dims(h[i-1], axis=0))
        dhnext = np.dot(Whh.T, dact)
    for dparam in [dLdWhx, dLdWhh, dLdWhy, dLdBy, dLdBh]:
      np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
    return total_loss,dLdWhx,dLdWhh,dLdWhy,dLdBy,dLdBh,h[len(inputs)-1]


In [0]:
def sample(hprevious, index, count):
    x = np.zeros((len(characters), 1))
    x[index] = 1
    predict = []
    hprevious=np.reshape(hprevious,(unit_size,1))
    for t in range(count):
        h = np.tanh(np.dot(Whx, x) + np.dot(Whh, hprevious)+Bh)
        y = np.dot(Why, h)
        p = np.exp(y) / np.sum(np.exp(y))
        predicted = np.random.choice(range(len(characters)), p=p.ravel())
        x = np.zeros((len(characters), 1))
        x[predicted] = 1
        predict.append(predicted)
    return predict

In [0]:
#Parameters upgrade, ı use Adagrad upgrade
# variables for previous gradient

pdLdWhx=np.zeros(Whx.shape) 
pdLdWhh=np.zeros(Whh.shape)
pdLdWhy=np.zeros(Why.shape)
pdLdBy=np.zeros(By.shape)
pdLdBh=np.zeros(Bh.shape)

#initial parameters 
    
#inputs=data_index[20:45]
#targets=data_index[21:46]  
unit_size=100
step_size=25
learning_rate=1e-2
hprevious=np.zeros((unit_size,1))


s,n = 0,0 # s start index for inputs, 


for i in range(1000000):


    if s>=850000:
       s=0

    inputs=data_index[s:s+step_size]
    targets=data_index[s+1:s+step_size+1]    
    s+=step_size+1
  
    
    total_loss,dLdWhx,dLdWhh,dLdWhy,dLdBy,dLdBh,hprevious=forward_prop(inputs,targets,hprevious)
    
    
    if n%5000==0:
      print('Total loss ', total_loss,'n',n,'s',s)
      a=sample(hprevious,inputs[0],120)
      print("".join([ix_to_char[ix] for ix in a]))
      print('\n')  

# Adagrad upgrade for paramaeters

    for parameters, dparam, pdparam in zip( [Whx, Whh, Why, Bh, By], 
                                            [dLdWhx, dLdWhh, dLdWhy, dLdBh, dLdBy], 
                                            [pdLdWhx, pdLdWhh, pdLdWhy, pdLdBh, pdLdBy]):
        pdparam += dparam * dparam
        parameters += -learning_rate * dparam / np.sqrt(pdparam + 1e-8) # 
        
    n=n+1
    

Total loss  39.733546810718984 n 0 s 26
isofoki’sofofofinofofofofoupevinousabofofifoudofofofofofofofofofofougofisoveva nofoupusoloupevani’soxalyoffofrimalexrave


Total loss  55.635727550105486 n 5000 s 130026
higffow w p angw ackep’dy jug w yigngngry ala w wedy ppp w blang’vedy w ca w sighigdemidy’vep y w wigongakedelllud w ex 


Total loss  38.14690021369081 n 10000 s 260026
on’rolasove’roxawaid holaly’landly’ten’lixay’chenono’ly k routilalisole’lyon’ly’lay’la’lalalalhinanonomanilay’lancile’la


Total loss  43.106595519326156 n 15000 s 390026
yofofoknofokli’sprabyosexexanoboofofofofofofofofofofofoknagofori bi’ni’sofi’sofagoknapofofokinofovabevexbyofofofofofaspe


Total loss  26.113111470976218 n 20000 s 520026
o’pesat chericto’st icechowenachicwerele’sto’mobengucccorpo'mo fy’shin miceaccofy’sict ichoweteten o’sto’rcat wo’stowhic


Total loss  38.87687381555088 n 25000 s 650026
moowofffyivicewe waqboto vice fobowace’ve’vice touchytobk gicbvotowo jice mimicowe ce fffuce’cetikebobaly we

## Results

I run my model 100 hidden unit  , 1e-1 learning rate and 25 charachter  memory . Total loss value  decreased 55 to 25 levels after  but model didn't provide reasonable text sample

Examples:

**Total loss  39.733546810718984 , Iteration 0**

"isofoki’sofofofinofofofofoupevinousabofofifoudofofofofofofofofofofougofisoveva nofoupusoloupevani’soxalyoffofrimalexrave "


**Total loss  30.808333850935334 Iteration 145000**

anl wilas choro ly’x wonhaworoubowuswanouthomou’ro’lay kay’xite’son’re’sowole’roroutouje’roulalache’rouhe’rocthe’rorout 


**Total loss  24.771582426613882 Iteration 220000**

"medjw tow yifw w kybearyormstif w k tw klytw clyic’sow k w’sw wex wi wmed p w tw’sow jwh’sowsswa tw wedmoc jw i ’tow a t"

**Total loss  32.26009262283761  Iteration 340000**
 
 "pice buqusowic wan re h tay ze’soxe mice hy’s ffowy m we way ch st ce icays wan p we pay’ssom owe sowy t so t ps j’s r ""





